In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-j812qe7y/unsloth_acd10f0cb487477ea6c9e9170d6e242e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-j812qe7y/unsloth_acd10f0cb487477ea6c9e9170d6e242e
  Resolved https://github.com/unslothai/unsloth.git to commit 7a8f99e1890213cdd01a3ab6c3e13174a96e8220
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 12.4 MB/s eta 0:00:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


### Background

I saw a reel on Instagram in which an AI enthusiast created an AI clone of himself to talk to his girlfriend (Certainly, I won't do that... xd) using [RAG](https://youtu.be/YVWxbHJakgg?feature=shared) Retrieval Augmented Generation and the Chat GPT-3.5 turbo API. It kind of worked, but it had major privacy issues. Sending personal chats to Chat GPT could potentially result in those chats being used by OpenAI to train its model. This led me to think, what if I fine-tuned a pre-existing model like Llama or Mixtral on my personal WhatsApp chat history? It would be cool to have a model that can talk like me as I do on WhatsApp, primarily in Hinglish (Hindi + English).

[Fine-tuning a large language model (LLM)](https://youtu.be/YVWxbHJakgg?feature=shared) requires a good understanding of LLMs in general. The major challenge with fine-tuning big models, such as a 7B parameter model, is that it requires a minimum of 32GB of GPU RAM, which is costly and not available in free-tier GPU compute services like Colab or Kaggle. So, I had to find a way around this limitation.

Another important aspect is that the fine-tuning results heavily depend on the quality and size of the dataset used. Converting raw WhatsApp chat data into a usable dataset is challenging but worth pursuing.

Let's see how it looks in reality and how it's being carried out.

### Important

We are using a free instance of Google Colab to fine-tune our model`(Llama3)`, making it **totally free**.

For chatting with our fine-tuned model, we will use [Ollama](https://ollama.com/) locally, which is very lightweight and requires only **8GB** of free RAM in your laptop/PC and works without any **GPU** support.

**Keep in mind that your chat data is completely safe; it is not being sent to anyone.**


### Data Prep
To extract chat history from your WhatsApp chats, follow these steps:

1. Open your WhatsApp application.
2. Go to the chat from which you want to extract the chat history.
3. Click on the three dots in the top right corner of the screen.
4. Click on `More` then click on `Export Chat`.
5. Select `Without media`.
6. Save it locally or send it to saved messages on Telegram so you can later download it on your Telegram desktop.
#### **7. Repeat these steps for all of your chats. The more chat data you have, the better the results will be.**

It will generate `.zip file`. You don't have to extract it.




#### Upload Exported Chat files to Colab runtime
Now, locate your exported chat zip files and upload them to the Colab runtime. Follow these steps to upload files to Google Colab:

1. Click on the Files icon on the left side of the screen (as shown in the image attached below).
2. Click on the upload button. It will open the File Explorer. Choose the exported chat zip files

##### **You can select multiple files at once**.

* Wait until your files are uploaded. The upload process bar will display at the bottom left corner of the screen.
* Once files are uploaded successfully, they will appear in the Files tab of Google Colab.

<img src="https://github.com/Eviltr0N/Make-AI-Clone-of-Yourself/raw/main/img/file_upload.png" width="480" height="600">

##### Keep in Mind:

* Export chat history only for meaningful conversations. Before exporting a chat, consider whether it adds value to the data or if it is just a short conversation.
* If you think you don’t want the AI to learn from a specific chat, then don’t export it.
#### **Currently, it only supports individual chats, so please do not export group chats.**


### Data Filtering

The exported data contains many irregularities such as `<Media omitted>`, `This message was deleted` and timestamps of messages. We need to remove these and convert the whole chat history into a `Prompt: Response` format so it can be used to fine-tune the model. To extract messages from the data, I used `regex`. Additionally, I filtered out any links and emails from the chat data for obvious privacy reasons.

**Now, please edit the below list of `filler_words`**. These words may vary from person to person. Some examples are `Ok`, `Yup`, `Hmm`, `Han`. We need to remove these from the dataset because if we don't, the fine-tuned model will primarily learn these words and, in most cases, respond with them. For example, if you ask, "Where are you going?" the model might respond with something like "Ok" or "Hmm."

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!

In [22]:
filler_words = ["Ok", "Okay", "Yup", "Hmm"]
# Add or remove words from this list based on your personal usage.

chat_dir = "./"

In [23]:
import re
import os
import shutil
import csv

In [24]:
class Wh_Chat_Processor:
    def __init__(self):
        pass
    def open_chat_file(self, dir,filename):
        self.sender_name = filename.replace("WhatsApp Chat with ", "").replace(".txt", "")
        with open(os.path.join(dir,filename)) as f:
            chat_text = f.read()
        return chat_text

    def msg_filter_basic(self, chat_text):
        filtered = []
        pt = r' - ([^:]+): (.*?)(?=\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}\s*(?:AM|PM|am|pm)? - |$)'
        msgs = re.findall(pt, chat_text, re.DOTALL)
        for msg in msgs:
            line = msg[1]
            wh_default_filter = "Tap to learn more." in line or "<Media omitted>" in line
            website_filter = "https://" in line or "http://" in line
            mail_filter = "@gmail.com" in line
            deleted_msg_filter = "This message was deleted" in line or "You deleted this message" in line or "<This message was edited>" in line or "(file attached)" in line

            if not (wh_default_filter or website_filter or mail_filter or deleted_msg_filter):
                    filtered.append(msg)
        return filtered

    def process_chat(self, chat_data):
        merged_lines = []
        current_sender = None
        current_message = {}
        for line in chat_data:
            if not line:
                continue
            parts = line
            if len(parts) == 2:
                sender, message = parts
                if current_sender is None:
                    current_sender = sender
                    current_message[current_sender] = [message.strip()]
                elif sender == current_sender:
                    current_message[current_sender].append(message.strip())
                else:
                    merged_lines.append(current_message)
                    current_sender = sender
                    current_message = {current_sender: [message.strip()]}
            else:
                if current_sender:
                    current_message[current_sender][-1] += " " + line.strip()
        if current_sender:
            merged_lines.append(current_message)
        keys = set()
        for line in merged_lines:
            # print(line)
            for key in line.keys():
                if key != self.sender_name:
                    keys.add(key)
        self.my_name = list(keys)[0]
        print(list(keys))
        return merged_lines

    def advance_filter(self, merged_chat_data):
        filtered_data=[]
        sender = ""
        me = ""
        chk = 1
        CD = merged_chat_data
        for ind, x in enumerate(CD):
            if x.get(self.sender_name) != None :
                if len(x[self.sender_name]) == 1 and ( x[self.sender_name][0] in filler_words or len(x[self.sender_name][0]) ==1 ):
                    continue
                if len(CD[ind][self.sender_name]) > 1:
                    for y in range(0,len(CD[ind][self.sender_name])):
                        if y+1 != len(CD[ind][self.sender_name]):
                            sender += CD[ind][self.sender_name][y] + "\n"
                        else:
                            sender += CD[ind][self.sender_name][y]
                else:
                    sender += CD[ind][self.sender_name][0]
            elif x.get(self.my_name) != None and len(sender) > 1:
                if len(CD[ind][self.my_name]) > 1:
                    for y in range(0,len(CD[ind][self.my_name])):
                        if y+1 != len(CD[ind][self.my_name]):
                            me += CD[ind][self.my_name][y] + "\n"
                        else:
                            me += CD[ind][self.my_name][y]
                else:
                    me += CD[ind][self.my_name][0]
            else:
                continue
            if chk ==1:
                chk+=1
            elif chk ==2:
                filtered_data.append([sender, me])
                sender = ""
                me=""
                chk=1
            else:
                pass
        return filtered_data


In [25]:
with open("all_chat_data.csv", "w") as f:
    f.write("Prompt,Response"+ "\n")

for file in os.listdir(os.path.join(chat_dir)):
    if file.endswith('.zip'):
        full_path = os.path.join(chat_dir, file)
        shutil.unpack_archive(full_path, chat_dir)

In [26]:
for file in os.listdir(os.path.join(chat_dir)):
    processor = Wh_Chat_Processor()
    if file.endswith('.txt'):
        print("Processing: ",file)
        chat_d = processor.open_chat_file(chat_dir,file)
        basic_f = processor.msg_filter_basic(chat_d)
        chat_ps = processor.process_chat(basic_f)
        filtered_data = processor.advance_filter(chat_ps)
        with open("all_chat_data.csv", "a") as f:
            csv_writer = csv.writer(f)
            for row in filtered_data:
                csv_writer.writerow(row)
print("Successfully Processed all the chats... Generated CSV File of chats is saved in Current directory with the name 'all_chat_data.csv'")

Processing:  WhatsApp Chat with Ankita.txt
['🧐🧐']
Processing:  WhatsApp Chat with Laxmi___🍁.txt
['🧐🧐']
Processing:  WhatsApp Chat with Rupes Bhaiya ❤️❤️.txt
['🧐🧐']
Processing:  WhatsApp Chat with Rahul Bhaiya ❤️❤️.txt
['🧐🧐']
Processing:  WhatsApp Chat with Richa.txt
['🧐🧐']
Processing:  WhatsApp Chat with Shivam.txt
['🧐🧐']
Successfully Processed all the chats... Generated CSV File of chats is saved in Current directory with the name 'all_chat_data.csv'


### Model Fine-Tuning
As we discussed earlier, fine-tuning a 7B parameter model with just 16GB of RAM is not possible. To achieve this, we will use a technique known as [Quantization](https://huggingface.co/docs/optimum/en/concept_guides/quantization). Specifically, we will use 4-bit quantization.

I am using [Unsloth](https://github.com/unslothai/unsloth) for the rest of the processes, such as quantization and training the model. Unsloth has very good documentation and requires less VRAM to fine-tune the model.

Check out - [Unsloth's Github](https://github.com/unslothai/unsloth)

This notebook uses the `Llama-3` format for conversation style finetunes. We use [Open Assistant conversations](https://huggingface.co/datasets/philschmid/guanaco-sharegpt-style) in ShareGPT style.

* Unsloth support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* Unsloth support 16bit LoRA or 4bit QLoRA. Both 2x faster.


For finetuning I am using **`Llama3` 8B Instruct** as our base model, you can use other models such as `Mixtral` and `Gemma`. I have traied Mixtral also but it dosent perform as good as `Llama3`.

In [27]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [28]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Let's prepare dataset from the filtered Whatsapp Chat data

In [29]:
import pandas as pd
from datasets import Dataset, load_dataset
from unsloth.chat_templates import get_chat_template

In [30]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3",  # Use the desired chat template
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"}
)

# Define the formatting function
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}


In [31]:
df = pd.read_csv("all_chat_data.csv")
conversations = []
for _, row in df.iterrows():
    try:
        conversation = [
            {'from': 'human', 'value': str(row['Prompt'])},
            {'from': 'assistant', 'value': str(row['Response'])}
        ]
        conversations.append(conversation)
    except:
        print(_ , row)


dataset = Dataset.from_dict({"conversations": conversations})
dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/293 [00:00<?, ? examples/s]

Let's see how the `Llama-3` format works by printing the 5th element

In [32]:
dataset[5]["conversations"]

[{'from': 'human', 'value': 'Tu bta na'},
 {'from': 'assistant', 'value': 'thode der wait kr le'}]

In [33]:
print(dataset[5]["text"])

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Tu bta na<|eot_id|><|start_header_id|>assistant<|end_header_id|>

thode der wait kr le<|eot_id|>



### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).
* I am doing `1 epochs` to speed things up, but you can set `num_train_epochs` to 2 or 3, Just experiment with it.
* If you have large dataset then just go for `1 full epoch`. Do not do more than 3 or 4 epoch if the `training loss` is not **decreasing**.

In [34]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/293 [00:00<?, ? examples/s]

#### Start Training

In [35]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 293 | Num Epochs = 1 | Total steps = 36
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: poojag4114 (poojag4114-st-andrews-institute-of-technology-and-management) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,1.907700
2,2.365800
3,4.199400
4,1.984600
5,1.711000
6,2.403700
7,1.324000
8,1.550000
9,0.722500
10,0.682400


Unsloth: Will smartly offload gradients to save VRAM!


<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `Llama-3`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [36]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
text_streamer = TextStreamer(tokenizer)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

output = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Who are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>





>

<|begin_of_text|><|begin_of_text|><|begin_of_text|>;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp;amp


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving finetuned model (LoRA adapter only)

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [37]:
model.save_pretrained("lora_model") # Local saving


### GGUF Conversion (For Ollama)
**To use our finetuned model in our PC/laptop we will use [Ollama](https://ollama.com/)**. To use this model with  `Ollama` We have to save the model in `GGUF` Format.

Unsloth allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on Unsloth [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
    * `q8_0` - Fast conversion. High resource use, but generally acceptable.
    * `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
    * `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [38]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.39 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 11.65it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:56<00:00,  3.65s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {4

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to model/Modelfile


Once the fine-tuning is complete, the model `unsloth.Q8_0.gguf` is saved in the `models/` folder. You need to download this `GGUF` file and `Modelfile` too. You can directly download it by locating the file in the files section of Google Colab, or you can copy this file to your Google Drive to download it from there. If your internet connection is slow like mine, the Google Drive method is best because the file is large (approximately 8GB). Here are the steps for both methods.

### Direct Download via Colab
1. Click on the files section in Google Colab.
2. Locate the models folder. Then expand it by clicking on the arrow located to the left of the folder name.
3. Choose the file `unsloth.Q8_0.gguf` & `Modelfile`, then hover the mouse cursor over the filename.
4. Click on the three dots, then select Download.

  <img src=https://github.com/Eviltr0N/Make-AI-Clone-of-Yourself/raw/main/img/file_download.png  width="400" height="700">

### Using Finetuned Model With Ollama and Whatsapp
Now follow this guide on my [Github](https://github.com/Eviltr0N/Make-AI-Clone-of-Yourself?tab=readme-ov-file#loading-the-model-into-ollama) to chat with Your finetuned model.
[Here](https://github.com/Eviltr0N/Make-AI-Clone-of-Yourself?tab=readme-ov-file#loading-the-model-into-ollama)

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
!mkdir /content/drive/MyDrive/finetuned_model
!cp /content/model/unsloth.Q8_0.gguf /content/drive/MyDrive/finetuned_model/
